# Multi-parameter Amp Model Training

## Step 1: Get Data

- Download the wave file to be played into the amp: [Proteus_Capture.wav](https://github.com/akaztp/Automated-GuitarAmpModelling-MultiParameter/raw/refs/heads/master/Data/Proteus_Capture.wav)
- Record the output from the amp with different values of its effects (i.e., parameters). Wave files are expected to be in 48kHz. Sample format can be any, but the higher the better like PCM24 or FP32.
- Generate a JSON file named "configuration.json" with the description of the recorded wave files
```JSON
{
    "Number of Parameters": 2,
    "Data Sets":[
    {
        "Parameters": [ 0.0, 0.0 ],
        "TrainingClean": "input.wav",
        "TrainingTarget": "0_0_output.wav"
    },
    {
        "Parameters": [ 0.0, 0.25 ],
        "TrainingClean": "input.wav",
        "TrainingTarget": "0_0.25_output.wav"
    },
    ...
    ]
}
```
Note: The parameters values are between 0.0 and 1.0
- Build a zip file with the "configuration.json" and the other wave files flat in the top level of the archive


In [ ]:
#@markdown ## Step 2: Setup environment, upload and extract training data
%cd /content
!rm -rf Automated-GuitarAmpModelling-MultiParameter
!git clone https://github.com/akaztp/Automated-GuitarAmpModelling-MultiParameter
%cd /content/Automated-GuitarAmpModelling-MultiParameter
!git submodule update --init --recursive
import os
# Set environment variable for protobuf to use pure-Python implementation
# This is slower but avoids compatibility issues with newer protobuf versions
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
!pip install -r requirements.txt --ignore-requires-python

# GPU Check
import torch
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

from google.colab import files
print("Please upload a ZIP file containing training configuration and wavefiles.")
uploaded = files.upload()
if uploaded:
    for filename in uploaded.keys():
            if filename.endswith('.zip'):
                !unzip {filename} -d input

In [ ]:
#@markdown ## Step 3: Prepare wav files for training
!python prep_wav2.py model -p "./input/configuration.json" # --normalize true

In [ ]:
#@markdown ## Step 4: Train with experiments and report
epochs = 200 #@param {type: "number"}
hidden_size1 = 15 #@param {type: "number"}
hidden_size2 = 20 #@param {type: "number"}
hidden_size3 = 30 #@param {type: "number"}
hidden_size4 = 40 #@param {type: "number"}
hidden_size5 = 50 #@param {type: "number"}
#@markdown Set any size to 0 to exclude it from the experiment

# Filter out zero values
hidden_sizes = [size for size in [hidden_size1, hidden_size2, hidden_size3, hidden_size4, hidden_size5] if size > 0]

# Import the experiment class
from hidden_size_experiment import HiddenSizeExperiment

# Create an experiment with custom hidden sizes
experiment = HiddenSizeExperiment(
    hidden_sizes=hidden_sizes,  # Your custom hidden sizes
    epochs=epochs,              # Number of epochs per model
)

# Run the experiment
results = experiment.run_experiment()

In [ ]:
#@markdown ## Step 5: Plot the result error-to-signal (optional)
hidden_size = 40 #@param {type: "number"}

# The plots will be generated in the Results/modelName/ directory
output = !python plot.py model --model_name hs_{hidden_size} --pred_wav test_out_best.wav

from IPython.display import display, Image
import re

match = re.search(r'NOTEBOOK_DISPLAY_IMAGES:(.*?),(.*?)$', '\n'.join(output), re.MULTILINE)

if match:
    comparison_path = match.group(1)
    detail_path = match.group(2)
    
    # Display the images
    print("\nSignal Comparison Plot:")
    display(Image(comparison_path))
    
    print("\nDetail Signal Comparison Plot:")
    display(Image(detail_path))
else:
    print("No image paths found in the output.")

In [ ]:
#@markdown ## Step 6: Download the results in a zip file.
hidden_size = 40 #@param {type: "number"}

!if [ -f results_hs_{hidden_size}.zip ]; then rm results_hs_{hidden_size}.zip; fi
!cd Results/hs_{hidden_size} && zip ../../results_hs_{hidden_size}.zip *
!cd ../../

# Download the zip file
from google.colab import files
files.download(f'results_hs_{hidden_size}.zip')


## Step 7:  Use the model in the plugin
The model file to use in the plugin is "model_best.json" from the downloaded archive.
For more than one parameter, identify the effect names and their order. This info is needed in the plugin.
Suggestion: rename the model json file to have the effects names, like "gibson-saturn_model_bass-treble.json"

# Notes and troubleshooting

- Make sure this notebook is connected and using GPU. There is a button "Connect" in the top right corner.  You have a limited number of consecutive GPU hours with the free version of Colab, but this will reset in a day or so. You might need to buy compute credits for more intensive work.
- The "Train" step can end before the specified number of epochs if the model stops improving enough. This step can also be stopped at any time.
- A model can be refined in more than one run of the "Train" step, by specifying more "epochs" and run the "Train" step again.
- The final loss values during training should be less than 0.08 to be considered successful. A loss of 0.02 or less is ideal. Higher than 0.10 indicates difficulty in training a model from the device, but note that mic'd amps will typically have a higher loss due to the complexities of the cab/mic. If the loss remains at 0.75 or higher, this could indicate a problem with the out.wav file, either from audio misalignment or error during the upload to Colab.
- For Parameterized Capture of an amp knob, if the device knob at 0% means there is no volume out of your device (such as a gain knob on an amplifier), then you should raise it slightly until your hear volume out. The model training doesn't do well if one of the target wavefiles is silent.


# Factors to Consider When Choosing Hidden Size

- Complexity of the Audio Transformation:
  - Higher complexity (like modeling complex distortion effects with many parameters) requires larger hidden sizes (40+)
  - Simpler transformations can work with smaller hidden sizes (20 or less)
- Available Training Data:
  - More training data supports larger hidden sizes without overfitting
  - Limited data may require smaller hidden sizes to prevent overfitting
- Computational Resources:
  - Larger hidden sizes require more memory and computational power
  - Consider your hardware limitations during both training and inference
- Real-time Performance Requirements:
  - If the model will be used in real-time applications, smaller hidden sizes will be more efficient
